
# Chapter 5: Colab에서 Ollama로 간단 Q&A

Colab 런타임에서 Ollama를 설치·구동하고, `llama3.1:8b` 모델로 사용자의 영어 질문을 한 번에 생성(비스트리밍)하여 출력합니다.


## 📚 학습 목표
- Ollama 로컬 REST API(`/api/version`, `/api/generate`) 호출 흐름 이해
- 최소 셋업(설치 → 서버 구동 → 모델 pull) 절차 습득
- 단일 질의 입력 → 비스트리밍 단일 응답 출력 패턴 익히기


## 📋 실습 구성
1) Google Colab 환경 설정: Ollama 설치, 서버 백그라운드 실행, 모델 pull
2) 헬스 체크: `/api/version`으로 서버 상태 확인
3) 헬퍼 함수: `ollama_generate(prompt)` 래퍼
4) 단일 생성: `input()`으로 질문 받고 즉시 응답 출력

> 참고: 본 예제는 RAG/대화/스트리밍/파라미터 튜닝을 제외한 최소 구성입니다.



---
## 1️⃣ 실행 전 준비

이 셀에서는 Colab 환경에서 Ollama를 설치하고 서버를 백그라운드로 구동한 뒤, 사용할 모델을 내려받습니다.

**주요 내용:**
- Ollama 설치 스크립트 실행
- `ollama serve` 백그라운드 구동(대기 포함)
- `llama3.1:8b` 모델 pull

**실행 결과:**
- 설치 완료 메시지와 함께 서버/모델 정보가 출력됩니다.
- 로컬 서버 엔드포인트: `http://localhost:11434`


In [ ]:
# ========================================
#  Google Colab 환경 설정
#    - Ollama 설치, 서버 백그라운드 실행, 모델 pull
# ========================================
import subprocess, time

# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# Ollama 서버 백그라운드 실행
print("Starting Ollama server ...")
ollama_process = subprocess.Popen(["ollama", "serve"],
                                  stdout=subprocess.DEVNULL,
                                  stderr=subprocess.DEVNULL)
# 서버 시작 대기
time.sleep(5)

# 모델 다운로드 (llama3.1:8b)
!ollama pull llama3.1:8b

print("\n✅ Setup done.")
print("   Ollama server: http://localhost:11434")
print("   Model: llama3.1:8b (non-streaming)")


---
## 2️⃣ Quick Health Check

이 셀에서는 Ollama 서버의 상태를 간단히 확인합니다.

**주요 내용:**
- `/api/version`에 GET 요청하여 정상 응답 여부 확인

**실행 결과:**
- 정상: 버전 JSON 출력(`{version: ...}`)
- 오류: 예외 메시지 문자열 출력

In [ ]:
# ========================================
# 2️⃣ Quick Health Check — /api/version
# ========================================
import requests

def ollama_ok(host: str = "http://localhost:11434"):
    try:
        r = requests.get(host + "/api/version", timeout=5)
        r.raise_for_status()
        return True, r.json()
    except Exception as e:
        return False, str(e)

ok, info = ollama_ok()
print("Server ready:", ok)
print("Version or error:", info)


---
## 3️⃣ Helper Functions

이 셀에서는 단일(turn) 생성용 래퍼 함수를 정의합니다.

**주요 내용:**
- `ollama_generate(prompt)`: `/api/generate`에 비스트리밍 요청
- 고정 파라미터(temperature/top_p/num_predict/repeat_penalty) 사용

**실행 결과:**
- 성공 시 `response` 텍스트 반환
- 실패 시 `[Error] ...` 메시지 반환

In [ ]:
# ========================================
# 3️⃣ Helper — Single-turn generation wrapper
#    - ollama_generate(prompt): 비스트리밍 단일 생성 호출
# ========================================
import requests, json

OLLAMA_HOST = "http://localhost:11434"
MODEL_NAME = "llama3.1:8b"

# Fixed defaults for simple non-streaming Q&A
_TEMPERATURE = 0.7
_TOP_P = 0.9
_NUM_PREDICT = 256
_REPEAT_PENALTY = 1.1

def ollama_generate(prompt: str) -> str:
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": _TEMPERATURE,
            "top_p": _TOP_P,
            "num_predict": _NUM_PREDICT,
            "repeat_penalty": _REPEAT_PENALTY,
        }
    }
    try:
        r = requests.post(f"{OLLAMA_HOST}/api/generate", json=payload, timeout=600)
        r.raise_for_status()
        data = r.json() if isinstance(r.json(), dict) else json.loads(r.text)
        return data.get("response", "")
    except Exception as e:
        return f"[Error] {e}"


---
## 4️⃣ Single-turn Generation

이 셀에서는 사용자의 영어 질문을 입력받아 단일 응답을 출력합니다.

**주요 내용:**
- `input()`으로 프롬프트를 받음
- `ollama_generate()`로 단일(비스트리밍) 생성 수행

**사용 방법:**
- 프롬프트에 영어 문장을 입력 후 Enter를 누르면 답변이 출력됩니다.
- 빈 입력 시 안내 메시지를 출력하고 종료합니다.

In [ ]:
# ========================================
# 4️⃣ Single-turn Generation — input() → generate → print
# ========================================
# Enter an English question and get a single response
user_prompt = input("Enter your question (English): ")
if user_prompt.strip():
    print(ollama_generate(user_prompt))
else:
    print("[Info] Empty prompt.")
